#   Dokumentacja klasyfikatora CIFAR-10 używając SVM

Autor: Filip Gębala

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

<h2>Model SVM</h2> <p>Maszyna wektorów nośnych (SVM, ang. Support Vector Machine) to nadzorowana metoda klasyfikacji, która znajduje optymalną hiperpłaszczyznę separującą dane w przestrzeni cech. Model ten stara się zmaksymalizować margines — odległość między hiperpłaszczyzną a najbliższymi punktami danych obu klas — co prowadzi do lepszej ogólności modelu i odporności na szum.</p> <p>W zastosowaniu do danych obrazowych z CIFAR-10, obrazy RGB są najpierw przekształcane do formatu tensora, a następnie spłaszczane do postaci wektorów cech. Ze względu na dużą wymiarowość danych (3072 cechy na obraz), przed klasyfikacją stosowana jest standaryzacja oraz redukcja wymiarowości metodą analizy głównych składowych (PCA). PCA redukuje dane do 300 wymiarów, co nie tylko przyspiesza trenowanie, ale także pomaga usunąć szum i zależności między cechami.</p> <p>Sam klasyfikator SVM wykorzystuje radialną funkcję bazową (RBF) jako jądro, co pozwala mu modelować nieliniowe granice decyzyjne. Model został wytrenowany z parametrem regularyzacji <code>C=10</code> oraz automatycznym dostrojeniem <code>gamma</code> do wartości „scale”, co zapewnia dobre dopasowanie przy umiarkowanym ryzyku przeuczenia. Trenowanie przeprowadzono na podzbiorze 100 000 próbek treningowych, a testowanie — na 1 000 próbkach z zestawu testowego CIFAR-10.</p> <p>Dzięki uprzedniej redukcji wymiarów oraz standaryzacji, klasyfikator osiąga przyzwoitą dokładność, biorąc pod uwagę ograniczenia związane z brakiem specjalistycznej ekstrakcji cech (jak w CNN). Model SVM dobrze sprawdza się jako klasyfikator bazowy lub komponent większych systemów klasyfikacyjnych, zwłaszcza w połączeniu z technikami redukcji wymiarów i przetwarzania wstępnego.</p>

<h2>Przygotowanie danych</h2>
<p>Podobnie jak w przypadku CNN, dane są ładowane przy użyciu `torchvision`. Jednak w tym przypadku nie stosuje się żadnej augmentacji ani normalizacji, ponieważ dane będą później przekształcane z użyciem `StandardScaler`. Dodatkowo, ponieważ pełny zbiór CIFAR-10 zawiera 50 tysięcy próbek, do treningu używamy podzbioru danych (np. 10 000), żeby skrócić czas treningu SVM. Każdy obraz zostaje spłaszczony do wektora, aby można było go użyć jako wejście do SVM.</p>

In [ ]:
# === TRANSFORMACJA ===
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                    (0.2023, 0.1994, 0.2010))
])

# === ZAŁADUJ ZBIÓR (trenuj na próbce) ===
trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

# Próbkuj mały podzbiór — pełen CIFAR-10 zajmuje za dużo czasu
def get_data(dataset, n_samples):
    loader = torch.utils.data.DataLoader(dataset, batch_size=n_samples, shuffle=True)
    images, labels = next(iter(loader))
    images = images.view(images.size(0), -1)  # flatten
    return images.numpy(), labels.numpy()

X_train, y_train = get_data(trainset, 100000)
X_test, y_test = get_data(testset, 1000)

<h2>Preprocessing: Standaryzacja i PCA</h2>
<p>Standaryzacja danych jest kluczowa przy stosowaniu SVM. Następnie przeprowadzana jest redukcja wymiarowości za pomocą PCA, co pozwala przyspieszyć uczenie i poprawia stabilność algorytmu. Zmniejszamy ilość wymiarów z 3072 do 300.</p>

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# === PCA ===
pca = PCA(n_components=300)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

<h2>Trenowanie modelu SVM</h2>
<p>SVM jest trenowane przy użyciu jądra RBF (gaussowskiego). Parametr `C=10` ustala stopień karania błędów klasyfikacji, a `gamma='scale'` automatycznie dobiera szerokość jądra RBF do danych. Trening może trwać kilka minut w zależności od liczby próbek i wydajności komputera.</p>

In [ ]:
# === TRENING ===
clf = svm.SVC(kernel='rbf', C=10, gamma='scale')
print("Trenowanie")
clf.fit(X_train, y_train)

# === TEST ===
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność na zbiorze testowym: {accuracy * 100:.2f}%")

<h2>Wyniki i podsumowanie </h2>
<p>SVM może osiągać całkiem dobre wyniki nawet bez głębokiego uczenia, szczególnie przy odpowiedniej redukcji wymiarowości (PCA) i dobrze dobranych parametrach. W porównaniu do CNN, SVM jest mniej podatne na przeuczenie, ale jego skuteczność może być ograniczona przez brak nieliniowych, wielowarstwowych reprezentacji danych.</p>

<p>W tym przypadku niestety SVM nie był dobrym wyborem - uzyskał dokładność na poziomie ~55%. Jest to spowodowane głównie dużym rozmiarem obrazów, co bardzo zniekształca hiperpłaszczyznę.</p>